In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 882.5/882.5 kB 29.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/Othercomputers/Desktop/SatellieteIMGDetection
import os
print(os.getcwd())



Mounted at /content/drive
/content/drive/Othercomputers/Desktop/SatellieteIMGDetection
/content/drive/Othercomputers/Desktop/SatellieteIMGDetection


In [ ]:

# 직접 YOLO 클래스를 임포트
from ultralytics import YOLO

# YOLO 모델 초기화
model = YOLO("yolo11x-obb.pt")
results = model.train(
    data="data_colab.yaml",
    epochs=350,
    batch=32,
    imgsz=256,
    patience=100,
    device="0",
    lr0 = 0.01,
    lrf = 0.1,
    single_cls = True,
    augment = True,
    optimizer = 'AdamW',
    seed = 44,
)



Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.8 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=obb, mode=train, model=yolo11x-obb.pt, data=data_colab.yaml, epochs=350, time=None, patience=100, batch=32, imgsz=256, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=44, deterministic=True, single_cls=True, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=F

100%|██████████| 755k/755k [00:00<00:00, 22.2MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      2784  ultralytics.nn.modules.conv.Conv             [3, 96, 3, 2]                 
  1                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  2                  -1  2    389760  ultralytics.nn.modules.block.C3k2            [192, 384, 2, True, 0.25]     
  3                  -1  1   1327872  ultralytics.nn.modules.conv.Conv             [384, 384, 3, 2]              
  4                  -1  2   1553664  ultralytics.nn.modules.block.C3k2            [384, 768, 2, True, 0.25]     
  5                  -1  1   5309952  ultralytics.nn.modules.conv.Conv             [768, 768, 3, 2]              
  6                  -1  2   5022720  ultralytics.nn.modules.block.C3k2            [768, 768, 2, True]           
  7                  -1  1   5309952  ultralytics

train: Scanning /content/drive/Othercomputers/Desktop/SatellieteIMGDetection/datasets/labels/train... 0 images, 485 backgrounds, 0 corrupt: 100%|██████████| 485/485 [00:09<00:00, 48.67it/s] 

train: WARNING ⚠️ No labels found in /content/drive/Othercomputers/Desktop/SatellieteIMGDetection/datasets/labels/train.cache. See https://docs.ultralytics.com/datasets for dataset formatting guidance.


...

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.18 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
val: Scanning /content/drive/Othercomputers/Desktop/SatellieteIMGDetection/datasets/labels/val... 0 images, 15 backgrounds, 0 corrupt: 100%|██████████| 15/15 [00:07<00:00,  2.10it/s]

val: WARNING ⚠️ No labels found in /content/drive/Othercomputers/Desktop/SatellieteIMGDetection/datasets/labels/val.cache. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
val: New cache created: /content/drive/Othercomputers/Desktop/SatellieteIMGDetection/datasets/labels/val.cache
WARNING ⚠️ No labels found in /content/drive/Othercomputers/Desktop/SatellieteIMGDetection/datasets/labels/val.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.


Plotting labels to runs/obb/train6/labels.jpg... 
zero-size array to reduction operation maximum which has no identity
optimizer: AdamW(lr=0.01, momentum=0.937) with parameter groups 173 weight(decay=0.0), 183 weight(decay=0.0005), 182 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 256 train, 256 val
Using 2 dataloader workers
Logging results to runs/obb/train6
Starting training for 350 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/350      6.48G          0      35.58          0          0        256: 100%|██████████| 16/16 [00:09<00:00,  1.64it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.74it/s]


RuntimeError: torch.cat(): expected a non-empty list of Tensors